In [2]:
import pandas as pandas
import numpy as np

You are absolutely correct. That is a very smart observation.Normalization (scaling values to 0-1 range) destroys the interpretability of the data.For Analysis: You need to know that PM2.5 is 45 µg/m³ (which is unhealthy), not 0.41 (which means nothing to a human).For Prediction: Neural Networks and Regression models calculate "gradients" and "weights". If one column is huge (e.g., Benzene = 700) and another is tiny (CO = 0.5), the model gets confused. So, we must normalize for prediction.The Strategy:We will create a "Master Cleaned Dataset" first. This dataset will have handled missing values but kept the original units.We use this Master Dataset for Analysis/Dashboard.We take a copy of this, normalize it, and feed it to the ML Model.Step 1: Basic Cleaning (The Foundation)Before we handle complex missing values, we need to fix the "structural" issues: bad column names, useless columns, and the wrong data type for Time.What we are doing here:Renaming: PM2.5 (Âµg/mÂ³) $\rightarrow$ PM2_5. (Coding with special characters is a nightmare; we fix that first).Dropping: Removing columns that are 99% empty (like Xylene).Datetime Conversion: Turning "2019-01-01" (text) into a real Date object so Python knows that "Jan 1" comes before "Jan 2".

In [9]:
File_name="C:\\Users\\Chinmaya holla\\Desktop\\Airpollution\\Data\\Unprocessed_data\\Consolidated_sheet_before_processing.xlsx"
df=pandas.read_excel(File_name,na_values=["NA"])
df



,Timestamp,PM2.5 (Âµg/mÂ³),PM10 (Âµg/mÂ³),NO (Âµg/mÂ³),NO2 (Âµg/mÂ³),NOx (ppb),NH3 (Âµg/mÂ³),SO2 (Âµg/mÂ³),CO (mg/mÂ³),Ozone (Âµg/mÂ³),...,AT (Â°C),RH (%),WS (m/s),WD (deg),RF (mm),TOT-RF (mm),SR (W/mt2),BP (mmHg),VWS (m/s),Station
0,2019-01-01,66.20,121.35,47.16,80.22,81.01,13.74,8.50,1.38,86.96,...,17.83,48.55,NaN,117.64,NaN,NaN,NaN,721.82,NaN,BapujiNagar
1,2019-01-02,52.06,100.73,40.41,77.26,73.95,13.56,8.00,1.11,77.22,...,13.63,51.41,NaN,96.77,NaN,NaN,NaN,722.08,NaN,BapujiNagar
2,2019-01-03,54.24,109.91,55.26,79.76,87.36,14.58,9.07,1.35,72.68,...,15.51,48.98,NaN,107.53,NaN,NaN,NaN,722.59,NaN,BapujiNagar
3,2019-01-04,56.64,113.99,51.41,82.45,85.65,15.13,8.62,1.20,59.44,...,15.60,45.80,NaN,105.88,NaN,NaN,NaN,721.78,NaN,BapujiNagar
4,2019-01-05,50.48,101.07,40.47,82.87,76.98,15.03,8.19,1.20,74.76,...,18.16,49.89,NaN,102.56,NaN,NaN,NaN,720.62,NaN,BapujiNagar
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17344,2024-12-27,16.87,36.64,1.33,14.38,8.72,1.90,7.28,0.47,19.78,...,20.12,68.55,0.99,99.92,NaN,0.0,NaN,718.44,NaN,Silkboard
17345,2024-12-28,26.02,68.69,1.36,14.46,8.80,1.83,8.47,0.44,21.35,...,20.72,61.74,1.06,95.00,NaN,0.0,NaN,718.19,NaN,Silkboard
17346,2024-12-29,28.00,65.62,1.39,14.14,8.65,1.71,8.23,0.21,21.59,...,18.29,66.00,0.98,90.12,NaN,0.0,NaN,718.38,NaN,Silkboard
17347,2024-12-30,18.96,68.04,1.28,14.38,8.70,1.77,7.49,0.50,21.23,...,25.04,58.88,1.39,105.71,NaN,0.0,NaN,717.00,NaN,Silkboard


In [15]:
new_columns = {
    'PM2.5 (Âµg/mÂ³)':'PM2_5',
    'PM10 (Âµg/mÂ³)':'PM10',
    'NO (Âµg/mÂ³)':'NO',
    'NO2 (Âµg/mÂ³)':'NO2',
    'NOx (ppb)':'NOx',
    'NH3 (Âµg/mÂ³)':'NH3',
    'SO2 (Âµg/mÂ³)':'SO2',
    'CO (mg/mÂ³)':'CO',
    'Ozone (Âµg/mÂ³)':'Ozone',
    'Benzene (Âµg/mÂ³)':'Benzene',
    'Toluene (Âµg/mÂ³)':'Toluene',
    'AT (Â°C)':'AT_C',
    'RH (%)':'RH_Percent',
    'WD (deg)':'WD_deg',
    'BP (mmHg)':'BP_mmHg'
}
df.rename(columns=new_columns,inplace=True)
columns_to_drop = [
    'Xylene (Âµg/mÂ³)', 'O Xylene (Âµg/mÂ³)', 'RF (mm)', 'VWS (m/s)', 
    'Eth-Benzene (Âµg/mÂ³)', 'MP-Xylene (Âµg/mÂ³)', 'TOT-RF (mm)', 
    'SR (W/mt2)', 'WS (m/s)'
]
df = df.drop(columns=columns_to_drop, errors='ignore')
df['Timestamp']=pandas.to_datetime(df['Timestamp'],errors='coerce')
df

,Timestamp,PM2_5,PM10,NO,NO2,NOx,NH3,SO2,CO,Ozone,Benzene,Toluene,AT_C,RH_Percent,WD_deg,BP_mmHg,Station
0,2019-01-01,66.20,121.35,47.16,80.22,81.01,13.74,8.50,1.38,86.96,0.10,0.46,17.83,48.55,117.64,721.82,BapujiNagar
1,2019-01-02,52.06,100.73,40.41,77.26,73.95,13.56,8.00,1.11,77.22,0.10,0.44,13.63,51.41,96.77,722.08,BapujiNagar
2,2019-01-03,54.24,109.91,55.26,79.76,87.36,14.58,9.07,1.35,72.68,0.10,0.48,15.51,48.98,107.53,722.59,BapujiNagar
3,2019-01-04,56.64,113.99,51.41,82.45,85.65,15.13,8.62,1.20,59.44,0.10,0.49,15.60,45.80,105.88,721.78,BapujiNagar
4,2019-01-05,50.48,101.07,40.47,82.87,76.98,15.03,8.19,1.20,74.76,0.10,0.44,18.16,49.89,102.56,720.62,BapujiNagar
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17344,2024-12-27,16.87,36.64,1.33,14.38,8.72,1.90,7.28,0.47,19.78,0.71,1.57,20.12,68.55,99.92,718.44,Silkboard
17345,2024-12-28,26.02,68.69,1.36,14.46,8.80,1.83,8.47,0.44,21.35,0.82,1.83,20.72,61.74,95.00,718.19,Silkboard
17346,2024-12-29,28.00,65.62,1.39,14.14,8.65,1.71,8.23,0.21,21.59,0.80,1.77,18.29,66.00,90.12,718.38,Silkboard
17347,2024-12-30,18.96,68.04,1.28,14.38,8.70,1.77,7.49,0.50,21.23,0.74,1.60,25.04,58.88,105.71,717.00,Silkboard


Step 2: Handling Missing Values (The "Interpolation" Technique)
Now we have clean columns, but the rows still have NaN (Not a Number) gaps.

Why not just replace with the Mean? If you replace a missing value in December with the yearly average, you are mixing Summer data into Winter. This destroys the seasonal patterns you are trying to find.

The Solution: Linear Interpolation We draw a straight line between the last known value and the next known value.

Monday: 10

Tuesday: NaN

Wednesday: 20

Interpolation fills Tuesday with 15.

This respects the trend of the data.

In [ ]:
# List of numeric columns to interpolate
pollutant_cols = [
    'PM2_5', 'PM10', 'NO', 'NO2', 'NOx', 'NH3', 'SO2', 'CO', 
    'Ozone', 'Benzene', 'Toluene', 'AT_C', 'RH_Percent', 'WD_deg', 'BP_mmHg'
]

# We must process each station SEPARATELY. 
df_imputed_list = []

for station in df['Station'].unique():
    # 1. Take one station's data
    df_station = df[df['Station'] == station].copy()
    
    # 2. Set Time as Index (Required for time-based interpolation)
    df_station = df_station.set_index('Timestamp')
    
    # 3. Resample to Daily ('D')
    # --- THE FIX IS HERE: numeric_only=True ---
    df_station_resampled = df_station.resample('D').mean(numeric_only=True)
    
    # 4. Interpolate (Fill gaps linearly based on time)
    df_station_resampled[pollutant_cols] = df_station_resampled[pollutant_cols].interpolate(method='time')
    
    # 5. Forward/Backward fill any remaining edge cases (start/end of dataset)
    df_station_resampled[pollutant_cols] = df_station_resampled[pollutant_cols].ffill().bfill()
    
    # 6. Restore the Station Name (it was lost because we excluded non-numerics)
    df_station_resampled['Station'] = station
    
    # 7. Reset index to make Timestamp a column again
    df_station_resampled = df_station_resampled.reset_index()
    
    df_imputed_list.append(df_station_resampled)

# Combine all stations back together
df_clean = pandas.concat(df_imputed_list)




Original Row Count: 17349
New Row Count (after fixing time gaps): 17779
Missing Values Remaining:
 Timestamp        0
PM2_5            0
PM10             0
NO            2192
NO2              0
NOx           2192
NH3              0
SO2              0
CO               0
Ozone            0
Benzene          0
Toluene       2192
AT_C          4384
RH_Percent       0
WD_deg        2192
BP_mmHg       2192
Station          0
dtype: int64


In [21]:
df_clean.to_csv('../data/Processed_data/master_cleaned_data.csv', index=False)

In [23]:
# Define the function to calculate AQI based on US EPA standards
def calculate_pm25_aqi(pm25):
    # Breakpoints for PM2.5
    c_low =    [0.0,   12.1,  35.5,  55.5,  150.5, 250.5, 350.5]
    c_high =   [12.0,  35.4,  55.4,  150.4, 250.4, 350.4, 500.4]
    i_low =    [0,     51,    101,   151,   201,   301,   401]
    i_high =   [50,    100,   150,   200,   300,   400,   500]
    
    if pandas.isna(pm25):
        return np.nan
    
    # Cap values > 500
    if pm25 > 500.4:
        return 500
        
    for i in range(len(c_low)):
        if c_low[i] <= pm25 <= c_high[i]:
            aqi = ((i_high[i] - i_low[i]) / (c_high[i] - c_low[i])) * (pm25 - c_low[i]) + i_low[i]
            return round(aqi)
            
    return np.nan

# Apply this function to our clean data
print("Calculating AQI... this might take a second.")
df_clean['PM2_5_AQI'] = df_clean['PM2_5'].apply(calculate_pm25_aqi)

# Save AGAIN. This is now your "Analysis Ready" file.
df_clean.to_csv('../data/Processed_data/analysis_ready_data.csv', index=False)

print("Done! Created 'PM2_5_AQI' column.")
print("Saved final file to 'data/Processed_data/analysis_ready_data.csv'")
print(df_clean[['Timestamp', 'Station', 'PM2_5', 'PM2_5_AQI']].head())

Calculating AQI... this might take a second.
Done! Created 'PM2_5_AQI' column.
Saved final file to 'data/Processed_data/analysis_ready_data.csv'
   Timestamp      Station  PM2_5  PM2_5_AQI
0 2019-01-01  BapujiNagar  66.20      157.0
1 2019-01-02  BapujiNagar  52.06      142.0
2 2019-01-03  BapujiNagar  54.24      147.0
3 2019-01-04  BapujiNagar  56.64      152.0
4 2019-01-05  BapujiNagar  50.48      138.0
